# YOLOv9 - O Estado da Arte para Detecção de Objetos

A detecção de objetos é uma tarefa fundamental em visão computacional, e tem visto um progresso extraordinário devido ao avanço dos modelos de *deep learning*.

Nesse escopo de problemas, a família YOLO (*You Only Look Once*) se destaca na vanguarda, reconhecida por sua excepcional velocidade de inferência em tempo real e precisão.

Se você ainda estava absorvendo e testando as aplicações da YOLOv8, eu tenho uma novidade para você. De maneira quase que inesperada, fomos presenteados com a novidade de uma YOLOv9 - possivelmente assumindo a posição de modelo no estado da arte.

<center>
<img src="https://sigmoidal.ai/wp-content/uploads/2024/02/yolov9-banner.png" width=600>
</center>


## *YOLOv9: Learning What You Want to Learn Using Programmable Gradient Information - Wang, Chien-Yao  and Liao, Hong-Yuan Mark (2024)*

Abordagens existentes de deep learning em detecção de objetos frequentemente enfatizam o design de arquiteturas de rede complexas ou a elaboração de funções objetivo especializadas. No entanto, elas tendem a negligenciar um problema crucial: os dados sofrem significativa perda de informação durante sua jornada pelas camadas da rede.

Este "gargalo de informação" corrompe os fluxos de gradiente durante a retropropagação, potencialmente levando a atualizações tendenciosas e predições imprecisas do modelo.

<center>
<img src="https://github.com/carlosfab/yolov9/raw/main/figure/performance.png" width=500></center>

Para enfrentar este desafio, os autores do paper propuseram o conceito de Informação de Gradiente Programável (PGI). PGI introduz um ramo reversível auxiliar que gera gradientes confiáveis, preservando características cruciais de entrada essenciais para a tarefa alvo. Ele evita a diluição semântica que pode ocorrer em técnicas tradicionais de supervisão profunda. Ao efetivamente "programar" o fluxo de informação de gradiente através da rede, PGI facilita resultados de treinamento ideais.

Além disso, os autores também introduziram a Rede de Agregação de Camadas Eficiente Generalizada (GELAN). GELAN equilibra meticulosamente a contagem de parâmetros, complexidade computacional, precisão e velocidade de inferência, oferecendo flexibilidade para implantação em diversos dispositivos. Em um feito notável, GELAN alcança utilização de parâmetros superior em comparação até mesmo com modelos de ponta que dependem de convoluções depth-wise.

**A sinergia de PGI e GELAN forma a espinha dorsal do YOLOv9.** Experimentos exaustivos no dataset MS COCO demonstram o domínio do YOLOv9. Especificamente:

* Desempenho Inigualável: YOLOv9 supera todos os detectores de objetos em tempo real existentes em métricas-chave.
* Eficiência: Supera até mesmo modelos pré-treinados em grandes conjuntos de dados, enquanto é treinado do zero.
* Leveza: PGI capacita modelos leves, tornando a detecção de objetos acessível para aplicações cotidianas.


Esta aula é apenas uma introdução sobre o modelo YOLOv9. A verdade é que toda a comunidade ainda está estudando o artigo, e validando as informações que têm chegado em um ritmo acelerado.

Se você quer se manter à frente no mercado de Visão Computacional, acompanhe esta primeira introdução que eu preparei para você!

In [16]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


Para clonar o repositório do YOLOv9 e instalar as dependências necessárias, siga os passos abaixo:


In [4]:
# Clona o repositório do YOLOv9
!git clone https://github.com/carlosfab/yolov9.git

# Muda o diretório de trabalho atual para o repositório YOLOv9 clonado
%cd yolov9

# Instala as dependências necessárias do YOLOv9 a partir do arquivo requirements.txt
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 168, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 168 (delta 27), reused 22 (delta 18), pack-reused 127
Receiving objects: 100% (168/168), 620.20 KiB | 9.40 MiB/s, done.
Resolving deltas: 100% (67/67), done.
/content/yolov9/yolov9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.1 MB/s eta 0:00:00


In [9]:
# Importa bibliotecas necessárias
import sys
import requests
from tqdm.notebook import tqdm
from pathlib import Path
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams


# Configuração de diretórios para código e dados
CODE_FOLDER = Path("..").resolve()  # Diretório do código
WEIGHTS_FOLDER = CODE_FOLDER / "weights"  # Diretório para pesos do modelo
DATA_FOLDER = CODE_FOLDER / "data"  # Diretório para dados

# Cria os diretórios para pesos e dados, se não existirem
WEIGHTS_FOLDER.mkdir(exist_ok=True, parents=True)
DATA_FOLDER.mkdir(exist_ok=True, parents=True)

# Adiciona o diretório do código ao path do Python para importação de módulos
sys.path.append(str(CODE_FOLDER))

rcParams['figure.figsize'] = 15, 15
%matplotlib inline

In [7]:
# URLs dos arquivos de pesos
weight_files = [
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt"
]

# Itera sobre a lista de URLs para baixar os arquivos de pesos
for i, url in enumerate(weight_files, start=1):
    filename = url.split('/')[-1]
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True, desc=f"Baixando arquivo {i}/{len(weight_files)}: {filename}")
    with open(WEIGHTS_FOLDER / filename, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

Baixando arquivo 1/4: yolov9-c.pt:   0%|          | 0.00/103M [00:00<?, ?iB/s]

Baixando arquivo 2/4: yolov9-e.pt:   0%|          | 0.00/140M [00:00<?, ?iB/s]

Baixando arquivo 3/4: gelan-c.pt:   0%|          | 0.00/51.5M [00:00<?, ?iB/s]

Baixando arquivo 4/4: gelan-e.pt:   0%|          | 0.00/117M [00:00<?, ?iB/s]

In [14]:
# URL da imagem de teste
url = 'https://sigmoidal.ai/wp-content/uploads/2022/11/314928609_1293071608150779_8666358890956473002_n.jpg'

# Faz a requisição para obter a imagem
response = requests.get(url)

# Define o caminho do arquivo onde a imagem será salva dentro do DATA_FOLDER
image_path = DATA_FOLDER / "test_image.jpg"

# Salva a imagem no diretório especificado
with open(image_path, 'wb') as f:
    f.write(response.content)

In [15]:
!python {CODE_FOLDER}/detect.py --weights {WEIGHTS_FOLDER}/yolov9-e.pt --conf 0.1 --source {DATA_FOLDER}/test_image.jpg --device cpu

detect: weights=['/content/yolov9/weights/yolov9-e.pt'], source=/content/yolov9/data/test_image.jpg, data=../data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=cpu, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 26e3791 Python-3.10.12 torch-2.1.0+cu121 CPU

Fusing layers... 
Model summary: 1119 layers, 69470144 parameters, 0 gradients, 244.0 GFLOPs
image 1/1 /content/yolov9/data/test_image.jpg: 640x608 6 persons, 1 sports ball, 17 bottles, 4 cups, 2 potted plants, 1 dining table, 3 books, 5982.5ms
Speed: 4.2ms pre-process, 5982.5ms inference, 38.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to ../runs/detect/exp


In [17]:
!python {CODE_FOLDER}/detect.py --weights {WEIGHTS_FOLDER}/yolov9-e.pt --conf 0.1 --source {DATA_FOLDER}/rua_1.mp4 --device cpu

detect: weights=['/content/yolov9/weights/yolov9-e.pt'], source=/content/yolov9/data/rua_1.mp4, data=../data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=cpu, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 26e3791 Python-3.10.12 torch-2.1.0+cu121 CPU

Fusing layers... 
Model summary: 1119 layers, 69470144 parameters, 0 gradients, 244.0 GFLOPs
video 1/1 (1/217) /content/yolov9/data/rua_1.mp4: 384x640 5 persons, 5 cars, 2 trucks, 3883.7ms
video 1/1 (2/217) /content/yolov9/data/rua_1.mp4: 384x640 3 persons, 6 cars, 1 truck, 3781.6ms
video 1/1 (3/217) /content/yolov9/data/rua_1.mp4: 384x640 3 persons, 5 cars, 1 truck, 1 traffic light, 4466.1ms
video 1/1 (4/217) /content/yolov9/data/rua_1.mp4: 